# Crop Productivity Analysis 

## Setup and Data Preparation

### Environment Setup

In [1]:
from pathlib import Path
import warnings

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer

warnings.filterwarnings("ignore")

### Helper functions

In [2]:
def find_project_root(marker_file: str = "pyproject.toml"):
    for parent in Path().cwd().parents:
        if (parent / marker_file).exists():
            return parent
    raise FileNotFoundError(f"No project root found with marker file: {marker_file}")


PROJECT_ROOT = Path().cwd().parent.parent
DATA_PATH = PROJECT_ROOT / "data"
EVI_PATH = DATA_PATH / "EVI and Crop Land" / "EVI 2010-2025"
CROPLAND_PATH = DATA_PATH / "EVI and Crop Land" / "Crop Land"
BOUNDARIES_PATH = DATA_PATH / "Shapefiles"

In [3]:
def clean_names(df: pd.DataFrame) -> pd.DataFrame:
    """Standardize column names."""
    df = df.rename(columns=lambda col: col.strip().lower().replace(" ", "_"))
    return df


def preprocess_evi(evi_file: str | Path) -> pd.DataFrame:
    """Preprocess EVI CSV file."""
    evi_df = pd.read_csv(evi_file)

    metadata_cols = [
        col for col in evi_df.columns if "EVI" not in col and "system:index" not in col
    ]
    evi_df = (
        evi_df.rename(columns=lambda col: col[-14:] if col.endswith("_EVI") else col)
        .drop(columns=["system:index"])
        .melt(
            id_vars=metadata_cols,
            var_name="band_date",
            value_name="EVI",
        )
        .assign(
            date=lambda df: pd.to_datetime(
                df["band_date"].str.extract(r"(\d{4}_\d{2}_\d{2})")[0],
                format="%Y_%m_%d",
            ),
        )
    )
    return evi_df

In [4]:
gdp_quarterly_raw = pd.read_excel(
    DATA_PATH / "GDP" / "Quarterly GDP per Sector.xlsx",
    sheet_name=1,
    skiprows=1,
    skipfooter=1,
)
gdp_adm1_raw = pd.read_excel(
    DATA_PATH / "GDP" / "Quarterly GDP per Sector.xlsx", sheet_name=0, skiprows=1
).rename(columns={"Unnamed: 0": "ADM1_NAME"})

evi_adm0_raw = (
    pd.concat(
        [
            preprocess_evi(
                DATA_PATH
                / "EVI and Crop Land"
                / "EVI 2010-2025"
                / "Admin level 0"
                / "MIMU"
                / f"myanmar_adm0_evi_stats_{year}.csv"
            )
            for year in range(2010, 2026)
        ],
    )
    .drop(columns=[".geo"])
    .sort_values(["date"])
    .reset_index(drop=True)
    .set_index("date")
)
evi_adm1_raw = (
    pd.concat(
        [
            preprocess_evi(
                DATA_PATH
                / "EVI and Crop Land"
                / "EVI 2010-2025"
                / "Admin level 1"
                / "MIMU"
                / f"myanmar_adm1_evi_stats_{year}_batch{batch}.csv"
            )
            for year in range(2010, 2026)
            for batch in range(1, 4)
        ],
    )
    .drop(columns=[".geo"])
    .sort_values(["ST", "date"])
    .reset_index(drop=True)
    .rename(columns={"ST": "adm1_name"})
    .set_index("date")
)

evi_adm0 = (
    evi_adm0_raw.groupby(pd.Grouper(freq="QS")).agg({"EVI": "median"}).pipe(clean_names)
)

evi_adm1 = (
    evi_adm1_raw.groupby(["adm1_name", pd.Grouper(freq="QS")])
    .agg({"EVI": "median"})
    .pipe(clean_names)
)

In [5]:
rainfall_adm0 = (
    pd.read_csv(
        DATA_PATH
        / "Rainfall"
        / "myanmar_chirps_rainfall_2012-01-01_2025-09-30_monthly_adm0.csv"
    )
    .assign(date=lambda df: pd.to_datetime(df["date"]))
    .set_index("date")
)

rainfall_adm1 = (
    pd.read_csv(
        DATA_PATH
        / "Rainfall"
        / "myanmar_chirps_rainfall_2012-01-01_2025-09-30_monthly_adm1.csv"
    )
    .assign(date=lambda df: pd.to_datetime(df["date"]))
    .rename(columns={"region": "adm1_name"})
    .set_index(["date", "adm1_name"])
)

## EVI and Economic Indicators

In [6]:
ntl_adm0_raw = pd.read_csv(DATA_PATH / "NTL" / "ntl_monthly_adm0_collection2.csv").drop(
    columns=["Unnamed: 0", "Unnamed: 0.1", "geometry"]
)
ntl_adm1_raw = pd.read_csv(DATA_PATH / "NTL" / "ntl_monthly_adm1_collection2.csv").drop(
    columns=["Unnamed: 0", "Unnamed: 0.1", "geometry"]
)

ntl_adm0 = (
    ntl_adm0_raw.assign(date=lambda df: pd.to_datetime(df["date"]))
    .pipe(clean_names)
    .set_index("date")
    .groupby(pd.Grouper(freq="QS"))
    .agg({"ntl_mean": "mean", "ntl_sum": "sum"})
    .assign(
        ntl_sum_lag_1=lambda df: df["ntl_sum"].shift(-1),
        ntl_sum_lag_2=lambda df: df["ntl_sum"].shift(-2),
    )
)

ntl_adm1 = (
    ntl_adm1_raw.assign(date=lambda df: pd.to_datetime(df["date"]))
    .pipe(clean_names)
    .set_index("date")
    .rename(columns={"adm1_en": "adm1_name"})
    .groupby(["adm1_name", pd.Grouper(freq="QS")])
    .agg({"ntl_mean": "mean", "ntl_sum": "sum"})
)

In [7]:
gdp_adm1_name_map = {
    "Ayeyarwaddy Region": "Ayeyarwady",
    "Bago Region": "Bago",
    "Chin State": "Chin",
    "Kachin State": "Kachin",
    "Kayah State": "Kayah",
    "Kayin State": "Kayin",
    "Magwe Region": "Magway",
    "Mandalay Region": "Mandalay",
    "Mon State": "Mon",
    "Nay Pyi Taw Council": "Nay Pyi Taw",
    "Rakhine State": "Rakhine",
    "Sagaing Region": "Sagaing",
    "Shan State": "Shan",
    "Tanintharyi Region": "Tanintharyi",
    "Yangon Region": "Yangon",
}


def preprocess_bago_and_shan(df: pd.DataFrame) -> pd.DataFrame:
    """Split the values of Bago into two equal parts and Shan into three equal parts."""
    df_bago = df.query("adm1_name == 'Bago'").copy()
    df_bago_new = pd.DataFrame(
        {
            "adm1_name": ["Bago (East)", "Bago (West)"],
            "agriculture": [df_bago["agriculture"].values[0] / 2] * 2,
            "industry": [df_bago["industry"].values[0] / 2] * 2,
            "services": [df_bago["services"].values[0] / 2] * 2,
        }
    )

    df_shan = df.query("adm1_name == 'Shan'").copy()
    df_shan_new = pd.DataFrame(
        {
            "adm1_name": ["Shan (South)", "Shan (East)", "Shan (North)"],
            "agriculture": [df_shan["agriculture"].values[0] / 3] * 3,
            "industry": [df_shan["industry"].values[0] / 3] * 3,
            "services": [df_shan["services"].values[0] / 3] * 3,
        }
    )

    df_rest = df.query("adm1_name not in ['Bago', 'Shan']").copy()

    return pd.concat([df_rest, df_bago_new, df_shan_new], ignore_index=True)


gdp_adm1 = (
    gdp_adm1_raw.pipe(clean_names)
    .assign(adm1_name=lambda df: df["adm1_name"].map(gdp_adm1_name_map))
    .pipe(preprocess_bago_and_shan)
    .assign(agriculture_pct=lambda df: df["agriculture"] / df["agriculture"].sum())
)

gdp_quarterly = (
    gdp_quarterly_raw.pipe(clean_names)
    .melt(
        id_vars=["quarter", "sub_group", "economic_activity"],
        var_name="year",
        value_name="gdp",
    )
    .assign(
        quarter_clean=lambda df: df["quarter"].str.strip(),
        year_first=lambda df: df["year"].str.split("-").str[0].str.strip(),
        year_last=lambda df: df["year"].str.split("-").str[1].str.strip(),
        # Map fiscal quarters to calendar quarters and years
        calendar_quarter=lambda df: df["quarter_clean"].map(
            {"Q1": "Q2", "Q2": "Q3", "Q3": "Q4", "Q4": "Q1"}
        ),
        year_selected=lambda df: df.apply(
            lambda row: row["year_last"]
            if row["quarter_clean"] == "Q4"
            else row["year_first"],
            axis=1,
        ),
        # year_selected=lambda df: df.apply(
        #     lambda row: row["year_last"]
        #     if row["quarter_clean"] in ["Q1", "Q2"]
        #     else row["year_first"],
        #     axis=1,
        # ),
        date=lambda df: pd.to_datetime(df["year_selected"] + df["calendar_quarter"]),
        sub_group=lambda df: df["sub_group"].str.strip(),
        economic_activity=lambda df: df["economic_activity"].str.strip(),
    )
    .query('sub_group == "Agriculture" and economic_activity == "Agriculture"')
    .set_index("date")
    .sort_index()
    .groupby(["sub_group", pd.Grouper(freq="QS")])
    .agg({"gdp": "sum"})
    .reset_index()
    .sort_values(["sub_group", "date"])
    .assign(
        gdp_lag_1=lambda df: df["gdp"].shift(1),
        gdp_lag_2=lambda df: df["gdp"].shift(2),
    )
)

gdp_quarterly_adm1 = (
    pd.DataFrame(
        [
            (region, period)
            for region in gdp_adm1["adm1_name"].unique()
            for period in gdp_quarterly["date"].unique()
        ],
        columns=["adm1_name", "date"],
    )
    .merge(
        gdp_adm1.filter(["adm1_name", "agriculture_pct"]), on="adm1_name", how="left"
    )
    .merge(gdp_quarterly, on="date", how="left")
    .rename(
        columns={
            "gdp": "gdp_total",
            "gdp_lag_1": "gdp_lag_1_total",
            "gdp_lag_2": "gdp_lag_2_total",
        }
    )
    .assign(
        gdp=lambda df: df["gdp_total"] * df["agriculture_pct"],
        gdp_lag_1=lambda df: df["gdp_lag_1_total"] * df["agriculture_pct"],
        gdp_lag_2=lambda df: df["gdp_lag_2_total"] * df["agriculture_pct"],
    )
    .set_index(["adm1_name", "date"])
    .filter(["gdp", "gdp_lag_1", "gdp_lag_2"])
)

In [8]:
evi_indicators = (
    gdp_quarterly.set_index("date")
    .join(evi_adm0, on="date", how="left")
    .join(ntl_adm0, on="date", how="left")
    .join(rainfall_adm0, on="date", how="left")
    .reset_index()
    .assign(
        evi_log=lambda df: np.log(df["evi"]),
        gdp_log=lambda df: np.log(df["gdp"]),
        ntl_sum_log=lambda df: np.log(df["ntl_sum"]),
        rainfall_mm_log=lambda df: np.log(df["rainfall_mm"]),
        evi_lag_1=lambda df: df["evi"].shift(1),
        evi_lag_2=lambda df: df["evi"].shift(2),
        rainfall_lag_1=lambda df: df["rainfall_mm"].shift(1),
        rainfall_lag_2=lambda df: df["rainfall_mm"].shift(2),
        evi_lag_1_log=lambda df: np.log(df["evi_lag_1"]),
        evi_lag_2_log=lambda df: np.log(df["evi_lag_2"]),
        rainfall_lag_1_log=lambda df: np.log(df["rainfall_lag_1"]),
        rainfall_lag_2_log=lambda df: np.log(df["rainfall_lag_2"]),
        ntl_sum_lag_1=lambda df: df["ntl_sum"].shift(1),
        is_crop_season=lambda df: df["date"].dt.month.isin([7, 8, 9, 10, 11, 12, 1, 2]),
    )
)

evi_adm0_annual = (
    evi_adm0_raw.groupby(pd.Grouper(freq="YS")).agg({"EVI": "median"}).pipe(clean_names)
)

The table below presents regression results for national median EVI and agricultural GDP. Lagged EVI (one quarter) is statistically significant across all specifications. In the full model, a 1% increase in EVI corresponds to a 1.12% increase in agricultural GDP, holding other variables constant.

In [50]:
mod_1 = smf.ols("gdp_log ~ evi_log", data=evi_indicators).fit()
mod_2 = smf.ols(
    "gdp_log ~ ntl_sum_log + evi_log + evi_lag_1_log",
    data=evi_indicators,
).fit()
mod_3 = smf.ols(
    "gdp_log ~ ntl_sum_log + evi_log + evi_lag_1_log + evi_lag_2_log",
    data=evi_indicators,
).fit()
mod_4 = smf.ols(
    "gdp_log ~ ntl_sum_log + evi_log + evi_lag_1_log + is_crop_season",
    data=evi_indicators,
).fit()

mod_4 = smf.ols(
    "gdp_log ~ ntl_sum_log + evi_log + evi_lag_1_log + is_crop_season",
    data=evi_indicators,
).fit()
mod_5 = smf.ols(
    "gdp_log ~ ntl_sum_log + evi_log + evi_lag_1_log + is_crop_season + rainfall_lag_1_log",
    data=evi_indicators,
).fit()

models = Stargazer([mod_1, mod_2, mod_3, mod_4, mod_5])
models.custom_columns(
    [
        "Current EVI only",
        "Current + 1 Quarter Ago",
        "Current + 2 Quarters Ago",
        "Current + 1 Quarter Ago + Is Crop Season",
        "All variables",
    ],
    [1, 1, 1, 1, 1],
)
models.covariate_order(
    [
        "evi_log",
        "ntl_sum_log",
        "evi_lag_1_log",
        "evi_lag_2_log",
        "is_crop_season[T.True]",
        "rainfall_lag_1_log",
    ]
)
models.rename_covariates(
    {
        "evi_log": "EVI (log)",
        "ntl_sum_log": "NTL Sum (log)",
        "evi_lag_1_log": "EVI 1 Quarter Ago (log)",
        "evi_lag_2_log": "EVI 2 Quarter Ago (log)",
        "is_crop_season[T.True]": "Is Crop Season",
        "rainfall_lag_1_log": "Rainfall (in mm and log)",
    }
)
models

### State-level EVI and Agricultural GDP

In [10]:
evi_indicators_adm1 = (
    gdp_quarterly_adm1.join(evi_adm1, how="left")
    .join(ntl_adm1, how="left")
    .join(rainfall_adm1, how="left")
    .reset_index()
    .sort_values(["adm1_name", "date"])
    .assign(
        evi_log=lambda df: np.log(df["evi"]),
        gdp_log=lambda df: np.log(df["gdp"]),
        rainfall_log=lambda df: np.log(df["rainfall_mm"]),
        ntl_sum_log=lambda df: np.log(df["ntl_sum"]),
        is_crop_season=lambda df: df["date"].dt.month.isin([7, 8, 9, 10, 11, 12, 1, 2]),
    )
    .groupby("adm1_name")
    .apply(
        lambda df: df.assign(
            evi_lag_1=df["evi"].shift(1),
            evi_lag_2=df["evi"].shift(2),
            rainfall_lag_1=df["rainfall_mm"].shift(1),
            rainfall_lag_2=df["rainfall_mm"].shift(2),
            evi_lag_1_log=lambda df: np.log(df["evi_lag_1"]),
            evi_lag_2_log=lambda df: np.log(df["evi_lag_2"]),
            rainfall_lag_1_log=lambda df: np.log1p(df["rainfall_lag_1"]),
            rainfall_lag_2_log=lambda df: np.log1p(df["rainfall_lag_2"]),
            ntl_sum_lag_1=df["ntl_sum"].shift(1),
            ntl_sum_lag_2=df["ntl_sum"].shift(2),
        ),
    )
    .reset_index(drop=True)
)

The table presents regression results for state-level median EVI and agricultural GDP.

In [11]:
mod_1 = smf.ols("gdp_log ~ evi_log", data=evi_indicators_adm1).fit()
mod_2 = smf.ols("gdp_log ~ ntl_sum_log + evi_log", data=evi_indicators_adm1).fit()
mod_3 = smf.ols(
    "gdp_log ~ ntl_sum_log + evi_log + evi_lag_1_log",
    data=evi_indicators_adm1,
).fit()
mod_4 = smf.ols(
    "gdp_log ~ ntl_sum_log + evi_log + evi_lag_1_log + is_crop_season",
    data=evi_indicators_adm1,
).fit()
mod_5 = smf.ols(
    "gdp_log ~ ntl_sum_log + evi_log + evi_lag_1_log + is_crop_season + rainfall_lag_1_log",
    data=evi_indicators_adm1,
).fit()
mod_6 = smf.ols(
    "gdp_log ~ ntl_sum_log + evi_log + evi_lag_1_log + is_crop_season + C(adm1_name)",
    data=evi_indicators_adm1,
).fit()

models = Stargazer([mod_1, mod_2, mod_3, mod_4, mod_5])
models.custom_columns(
    [
        "Current EVI only",
        "Current + NTL",
        "Current + NTL + 1 Quarter Ago",
        "Current + NTL + 1 Quarter Ago + Is Crop Season",
        "All Variables + Region Fixed Effects",
    ]
)
models.covariate_order(
    ["Intercept", "evi_log", "evi_lag_1_log", "is_crop_season[T.True]", "ntl_sum_log"]
)
models.rename_covariates(
    {
        "evi_log": "EVI (log)",
        "evi_lag_1_log": "EVI 1 Quarter Ago (log)",
        "is_crop_season[T.True]": "Is Crop Season",
        "ntl_sum_log": "NTL Sum (log)",
    }
)
models